In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
%matplotlib inline
import os
import re
from aicsimageio import AICSImage
os.chdir('Path_To_Directory_With_TiffMask_and_TrackMate_SubDirectories')

In [2]:
# Define TrackMate_Quant function, can collapse code after running first time

def TrackMate_Quant(spots, img1, Z_um, Y_um, X_um, Pct_Threshold):
    
    # Set Threshold for each image
    Threshold = 200
    
    # Convert um positions back to pixel coordinates
    spots['Pixel_Z'] = spots.POSITION_Z / Z_um
    spots['Pixel_Y'] = spots.POSITION_Y / Y_um
    spots['Pixel_X'] = spots.POSITION_X / X_um
    
    # Round to integers Drop any puncta that lands on edge pixels (indexed Pixel must be > 0 and < -1 than length of the array)
    #print(f'Spots before edge filtering: {spots.shape}')
    spotsF = spots.loc[:,['Pixel_Z','Pixel_Y','Pixel_X']].round(decimals=0)
    spotsF = spotsF.loc[((spotsF.Pixel_Z < (len(img1[0])-1)) & (spotsF.Pixel_Z > 0))&
                          ((spotsF.Pixel_Y < (len(img1[0][0])-1)) & (spotsF.Pixel_Y > 0))&
                          ((spotsF.Pixel_X < (len(img1[0][0][0])-1)) & (spotsF.Pixel_X > 0)),:]
    #print(f'Spots after edge filtering: {spotsF.shape}')
    # Get numpy array of 3D coordinates [Z, Y, X], convert to np.int64
    spots_coord = spotsF.to_numpy()
    spots_coord_round = spots_coord.astype(np.int64)
    
    # Calcualte image volume and store (exlcuding all edge pixels, so must subtract 2 from each dimension)
    Pixel_Vol = Z_um*Y_um*X_um
    Total_Pixels = (len(img1[0])-2)*len(img1[0][0]-2)*len(img1[0][0][0]-2)
    Total_Vol =  Total_Pixels*Pixel_Vol
    Neurite_Pixels = np.count_nonzero(img1[0] > Threshold)
    Neurite_Vol = Neurite_Pixels*Pixel_Vol
    Neurite_Vol_Pct = Neurite_Vol/Total_Vol
    
    # Initialize empty list for output
    RNAlist = []
    for i in list(np.arange(0,len(spotsF))):
        Z = spots_coord_round[i][0]
        Y = spots_coord_round[i][1]
        X = spots_coord_round[i][2]
        RNAlist.append([
            img1[0][Z][Y][X],img1[0][Z][Y-1][X+1],img1[0][Z][Y][X+1],img1[0][Z][Y+1][X+1],img1[0][Z][Y-1][X-1],
            img1[0][Z][Y][X-1],img1[0][Z][Y+1][X-1],img1[0][Z][Y-1][X],img1[0][Z][Y+1][X],img1[0][Z+1][Y][X],
            img1[0][Z+1][Y][X+1],img1[0][Z+1][Y][X-1],img1[0][Z+1][Y-1][X],img1[0][Z+1][Y+1][X],img1[0][Z-1][Y][X],
            img1[0][Z-1][Y][X+1],img1[0][Z-1][Y][X-1],img1[0][Z-1][Y-1][X],img1[0][Z-1][Y+1][X]
                ])
        ##
    RNA_Pct_list = []
    for i in list(np.arange(0,len(spotsF))):
        RNA_Pct_list.append(sum(1 for x in RNAlist[i] if (x > Threshold))/len(RNAlist[i]))
    # Output
    SpotCount = len(RNAlist)
    SpotCount_Pos = sum(1 for x in RNA_Pct_list if (x > Pct_Threshold))
    if SpotCount > 0 :
        PosSpots_Percent = SpotCount_Pos/SpotCount
    else :
        PosSpots_Percent = 0
    SpotCount_Vol = SpotCount/Total_Vol
    SpotCount_Neurite_Vol = SpotCount/Neurite_Vol
    SpotCountPos_Vol = SpotCount_Pos/Total_Vol
    SpotCountPos_Neurite_Vol = SpotCount_Pos/Neurite_Vol
    #
    return [SpotCount, SpotCount_Pos, PosSpots_Percent, SpotCount_Vol, SpotCount_Neurite_Vol, SpotCountPos_Vol, SpotCountPos_Neurite_Vol, Total_Pixels, Total_Vol, Neurite_Pixels, Neurite_Vol, Neurite_Vol_Pct]

In [3]:
# Set directory for files
directory = r'Path_To_Directory_With_TiffMask_and_TrackMate_SubDirectories/'
Tif_Paths = []

#
for filename in os.listdir(f'{directory}TiffMasks/'):
    if filename.endswith(".tif"):
        Tifp = os.path.join(f'{directory}TiffMasks/', filename)
        Tif_Paths.append(Tifp)
#
Filebase = []
# Need to have the binary masks in a separate folder 'TiffMasks' here
pattern = "Path_To_Directory_With_TiffMask_and_TrackMate_SubDirectories/TiffMasks/(.*?).tif"
for i in list(np.arange(0,len(Tif_Paths))):
    s = Tif_Paths[i]
    substring = re.search(pattern, s).group(1)
    Filebase.append(substring[:-6])
    
# 
Masks = []
for i in list(np.arange(0,len(Tif_Paths))):
    s = Tif_Paths[i]
    substring = re.search(pattern, s).group(1)
    Masks.append(substring)

In [4]:
# YOU MUST ENTER THIS INFORMATION FROM TRACKMATE BEGINNING SCREEN IN ORDER TO CONVERT THE UM BACK TO PIXEL COORDINATES
# IT MUST BE THE SAME FOR ALL IMAGES / TRACKMATE SPOT OUTPUT TO BE ANALYZED
Z_um = 0.300
Y_um = 0.180
X_um = 0.180

# Set % of Neurite+ pixels threshold for spot calling
Pct_Threshold = 0.6

#  Initiliaze empty list to append output to
RNA1_DataList = []
RNA2_DataList = []

In [5]:
for i in list(np.arange(0,len(Tif_Paths))):
    img = AICSImage(Tif_Paths[i]) # Read in Tiff Image as 6D Numpy Array
    img1 = img.data[0][0]       # Strip extra dimensions (Scence and Time)
    #
    # Read in RNA1 Spots first, run TrackMate_Quant, append RNA1 data to list
    spots = pd.read_csv(f'{directory}/TrackMate/{Filebase[i]}_Spots_RNA1.csv',sep=',',index_col=0)   
    RNA1_store = TrackMate_Quant(spots, img1, Z_um, Y_um, X_um, Pct_Threshold)    
    RNA1_DataList.append(RNA1_store)
    # Read in RNA2 Spots next, run TrackMate_Quant, append RNA2 data to list
    spots = pd.read_csv(f'{directory}/TrackMate/{Filebase[i]}_Spots_RNA2.csv',sep=',',index_col=0)   
    RNA2_store = TrackMate_Quant(spots, img1, Z_um, Y_um, X_um, Pct_Threshold)              
    RNA2_DataList.append(RNA2_store)

In [6]:
RNA1_Spot_Data = pd.DataFrame(RNA1_DataList,columns=['SpotCount', 'SpotCount_Pos', 'PosSpots_Percent',
                                                 'SpotCount_Vol', 'SpotCount_Neurite_Vol', 'SpotCountPos_Vol', 'SpotCountPos_Neurite_Vol',
                                                 'Total_Pixels', 'Total_Vol', 'Neurite_Pixels', 'Neurite_Vol', 'Neurite_Vol_Pct'])
#
RNA2_Spot_Data = pd.DataFrame(RNA2_DataList,columns=['SpotCount', 'SpotCount_Pos', 'PosSpots_Percent',
                                                 'SpotCount_Vol', 'SpotCount_Neurite_Vol', 'SpotCountPos_Vol', 'SpotCountPos_Neurite_Vol',
                                                 'Total_Pixels', 'Total_Vol', 'Neurite_Pixels', 'Neurite_Vol', 'Neurite_Vol_Pct'])
#
RNA1_Spot_Data['RNA'] = 'RNA1'
RNA2_Spot_Data['RNA'] = 'RNA2'

RNA1_Spot_Data['Filebase'] = Filebase
RNA2_Spot_Data['Filebase'] = Filebase
RNA1_Spot_Data['Mask'] = Masks
RNA2_Spot_Data['Mask'] = Masks
RNA1_Spot_Data = RNA1_Spot_Data.sort_values('Mask').set_index('Filebase',drop=True).copy()
RNA2_Spot_Data = RNA2_Spot_Data.sort_values('Mask').set_index('Filebase',drop=True).copy()

In [8]:
#
PathInfo = pd.read_csv('NeuritePathInfo.csv',index_col=0)
RNA1_Spot_Data['Length'] = PathInfo.sort_values('Mask').Length
RNA2_Spot_Data['Length'] = PathInfo.sort_values('Mask').Length
RNA1_Spot_Data['Puncta_um'] = RNA1_Spot_Data.SpotCount_Pos/RNA1_Spot_Data.Length
RNA2_Spot_Data['Puncta_um'] = RNA2_Spot_Data.SpotCount_Pos/RNA2_Spot_Data.Length

#
MergePlot = pd.concat([RNA1_Spot_Data, RNA2_Spot_Data])
MergePlot['SpotCountPos_Vol_10um3'] = MergePlot.SpotCountPos_Vol * 1e3
MergePlot['SpotCountPos_Neurite_Vol_10um3'] = MergePlot.SpotCountPos_Neurite_Vol * 1e3


In [10]:
# Write Output
#MergePlot.to_csv('perNeurite_RNA1_RNA2_MergeOutput.csv',sep=',')